# Refine Chain

## Custom Refine Chain (building steps)

In [ ]:
from functools import partial
from operator import itemgetter

from langchain.callbacks.manager import trace_as_chain_group
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document

In [ ]:
# Chain for generating initial summary based on the first document

llm = ChatOpenAI()
first_prompt = PromptTemplate.from_template("Summarize this content:\n\n{context}")
document_prompt = PromptTemplate.from_template("{page_content}")
partial_format_doc = partial(format_document, prompt=document_prompt)
summary_chain = {"context": partial_format_doc} | first_prompt | llm | StrOutputParser()

In [ ]:
# Chain for refining an existing summary based on
# an additional document

refine_prompt = PromptTemplate.from_template(
    "Here's your first summary: {prev_response}. "
    "Now add to it based on the following context: {context}"
)
refine_chain = (
    {
        "prev_response": itemgetter("prev_response"), 
        "context": lambda x: partial_format_doc(x["doc"])
    } | refine_prompt
    | llm 
    | StrOutputParser()
)

In [ ]:
# The final refine loop, which generates an initial summary
# then iteratively refines it based on each of the rest of the documents

def refine_loop(docs):
    with trace_as_chain_group("refine loop", inputs={"input": docs}) as manager:
        summary = summary_chain.invoke(
            docs[0], 
            config={"callbacks": manager, "run_name": "initial summary"}
        )
        for i, doc in enumerate(docs[1:]):
            summary = refine_chain.invoke(
                {"prev_response": summary, "doc": doc}, 
                config={"callbacks": manager, "run_name": f"refine {i}"}
            )
        manager.on_chain_end({"output": summary})
    return summary

## Current approach (not using above building steps)

"query" --> *rewrite query to search query*
- "Ich möchte an einem Projekt zu Augmented Reality arbeiten" --> "augmented reality"

- similarity search, k=4
- get sources
- search again with filtered sources
- get initial and relevant chunks for each source
- for each source: refine chain with all chunks (calls llm)
- check generated answer for correctness (calls llm) 

**Imports**

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding_function = OpenAIEmbeddings(show_progress_bar=True)

**Load database**

In [3]:
vectordb = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)

**Get user query**

In [67]:
query = "wer setzt sich mit augmented reality auseinander?"

**Retrieved docs initial**

In [68]:
initial_docs = vectordb.similarity_search(query, k=4)

  0%|          | 0/1 [00:00<?, ?it/s]

**Get different sources**

In [70]:
sources = set()
for doc in initial_docs:
    sources.add(doc.metadata["source"])

print(sources)

{'acke', 'webw', 'weei'}


**For each source get all relevant chunks**
- first chunk contains name
- all others by semantic similarity to `query`
</br></br>
**ToDo**
  - change `k` for relevant_chunks per person

In [71]:
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.llms import OpenAI

# Load the refine QA chain
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", verbose=False)

# Define the input documents and the question
question = f"Antworte auf Deutsch. Warum wurde diese Person gefunden, als der User nach dieser Frage gesucht hat: {query}"

### For the final step

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0)

template_string = """Stelle sicher, dass der Text grammatikalisch korrekt und kohärent ist. \
Falls der letzte Satz unvollständig ist, entferne ihn. Füge keine Informationen hinzu. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

In [73]:
from langchain.schema import Document

In [74]:
# docs = []

for source in sources:
    docs = []
        
    # First chunk for every person containts their name and occupation
    initial_chunk = vectordb.get(where={"source": source})["documents"][0]
    
    docs.append(
            Document(
                page_content = initial_chunk,
                metadata = {"source": source}
            )
    )
    
    # All chunks for the person related to the user query
    docs += vectordb.similarity_search(query, k=5, filter={"source": source})

    # Run the refine chain for all docs per person
    result = chain({"input_documents": docs, "question": question}, return_only_outputs=True)

    # Retrieve the refined answer
    refined_answer = result["output_text"]

    # Make sure the answer is grammatically correct
    refined_prompt = prompt_template.format_messages(text=refined_answer)
    answer = chat(refined_prompt) 

    print(answer.content)

  0%|          | 0/1 [00:00<?, ?it/s]

Dr. Philipp Ackermann wurde gefunden, weil er an der ZHAW Dozent für Informatik im Schwerpunkt "Human-Centered Computing" ist, was ein Teilbereich der Augmented Reality ist. Er ist auch Gründer und ehemaliger Geschäftsführer der Perspectix AG und Mitglied in verschiedenen Netzwerken wie IEEE ACM und der Schweizerischen Gesellschaft für Medizinische Informatik (SGMI). Er hat an verschiedenen Projekten gearbeitet, wie zum Beispiel Mixed Reality im Anlagenbau, epidemiologische Multiskalensimulation zur Analyse der Übertragungsmechanismen in der COVID-19-Pandemie, intelligenter Weinbau, Erstellung digitaler Gesundheitsangebote für AXA-Kunden, datengetriebenes medizinisches Muskeltraining und DeepScore: ein digitales Notenpult mit musikalischem. Er hat auch ein Papier veröffentlicht.


  0%|          | 0/1 [00:00<?, ?it/s]

Prof. Dr. Wibke Weber wurde gefunden, weil sie Professorin für Medienlinguistik ist und sich mit visueller Kommunikation, Augmented Reality, Virtual Reality, Medienkonvergenz, Multimodalität, Comics Journalism, Informationsdesign, Radiojournalismus und anderen Themen beschäftigt. Sie hat Forschungsarbeiten zu AR und VR veröffentlicht, wie z.B. "Forschungsperspektiven auf AR und VR" (Weber, 2017), "Narrativity and beyond: visual storytelling in newsrooms" (Weber, 2017) und "Virtual reality as a tool for political decision-making? : an empirical study on the power of immersive images on voting behavior" (Weber et al., 2022). Sie hat auch Forschungsarbeiten zu visuellen Stilpraktiken in Geschäftsberichten veröffentlicht, wie z.B. "Design matters: visuelle Stilpraktiken in Geschäftsberichten" (Weber, 2020).


  0%|          | 0/1 [00:00<?, ?it/s]

Christian Weber ist ein wissenschaftlicher Mitarbeiter am Institut für Wirtschaftsinformatik der ZHAW School of Management and Law. Er hat Forschungsprojekte, Lehre und Weiterbildung im Bereich der Augmented Reality durchgeführt, einschließlich nachhaltiger smarter Lösungen für Menschen in digitalen Ökosystemen (Digital Health, Smart & Ambient Assisted Living, Smart City, usw.), Anwendung von Mixed-Reality-Lösungen (VR/AR) in computergestützten kooperativen Arbeits- und Lernszenarien des Medizinsektors, Datenschutz, Cybersecurity, Compliance und Forensik als Enabler für digitale Ökosysteme sowie Anwendungen von Open Source Systemen im betrieblichen und infrastrukturellen Kontext von KMUs. Er hat auch Kenntnisse mit Lehrbezug (Didaktik) wie Lernen 4.0 - M.
